## Travel Planning Agentic System using LLMchain

### This project is a simulated agentic workflow using LLMChain:
#### LLMChain = deterministic task executor (1 prompt -> 1 response)
#### Use SequentialChain to coordinate the "agents"
#### Each agent = 1 chain with its own prompt

For agents like "Itinerary Planner", where no tools are needed and you just want a clear response,
we do not require to use agents at all.
We can achieve the same thing with LLMChain. In this the agents were not explicitly defined as standalone objects.
Instead, LLMChain instances are used to perform the roles of agents.

Langchain has two levels of abstraction of AI_driven workflows:
1. LLMChain (these are not full "agents" - they are deterministic chains of prompt + LLM).
Suitable for sequential tasks. Not decision-making, tool selection, or memory
2. LangChain Agents (using this, we will define true agents. initialize_agents, AgentExecutor+tools, Tool use + memory + reasoning)

1.Setup & Environment

In [1]:
!pip install langchain langchain_openai langchain_community python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [2]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [30]:
# Load API Keys
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("Missing OpenAI or Tavily API keys")

In [31]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, openai_api_key=openai_api_key)

2. Prompt Templates for each LLMChain Role

In [32]:
# Destination Researcher
destination_prompt = PromptTemplate.from_template("""
Research comprehensive travel information about {destination}:
Include best seasons to visit, safety, etiquette, costs, language, transport options, etc.
Travel Dates: {travel_dates}, Budget: {budget}.
""")

# Attractions Specialist
attractions_prompt = PromptTemplate.from_template("""
List top attractions and experiences in {destination} for travelers who enjoy {preferences}.
Budget: {budget},
Trip Duration: {duration_days} days.
""")

# Itinerary Planner
itinerary_prompt = PromptTemplate.from_template("""
Created a detailed {duration_days}-day travel itinerary for {destination} starting {travel_dates}.
Make sure the itinerary reflects preferences like: {preferences}, and fits within a {budget} Budget.
""")

# Local Guide
guide_prompt = PromptTemplate.from_template("""
Provide local tips, cultural insights, and safety advice for visiting {destination}.
Tailor to interests: {preferences}.
Budget level: {budget}.
""")


3. Define LLMChain for each Role

In [33]:
destination_chain = LLMChain(llm=llm, prompt=destination_prompt, output_key="destination_report")
attractions_chain = LLMChain(llm=llm, prompt=attractions_prompt, output_key="attractions")
itinerary_chain = LLMChain(llm=llm, prompt=itinerary_prompt, output_key="itinerary")
guide_chain = LLMChain(llm=llm, prompt=guide_prompt, output_key="guide")

4. Take Input from User

In [34]:
destination = input("Destination: ")
travel_dates = input("Travel Dates (default June 1, 2023): ") or "June 1, 2023"
duration_days = int(input("Duration (default 3): ") or "3")
preferences = input("Preferences (default: Cultural & Museums, Food & Culinary): ") or "Cultural & Museums, Food & Culinary"
budget = input("Budget (default: moderate): ") or "moderate"

input_variables = {
    "destination": destination,
    "travel_dates": travel_dates,
    "duration_days": duration_days,
    "preferences": preferences,
    "budget": budget
}

Destination: chicago
Travel Dates (default June 1, 2023): 3
Duration (default 3): 
Preferences (default: Cultural & Museums, Food & Culinary): 
Budget (default: moderate): 


5. Use SequentialChain to Run All Chains in Order

In [37]:
travel_chain = SequentialChain(
    chains = [destination_chain, attractions_chain, itinerary_chain, guide_chain],
    input_variables=["destination", "travel_dates", "duration_days", "preferences", "budget"],
    output_variables=["destination_report", "attractions", "itinerary", "guide"],
    verbose=True
)

# Run the LLMChain-based agentic system
results = travel_chain.invoke(input_variables)



> Entering new SequentialChain chain...

> Finished chain.


6. Print the Final Travel Plan Output

In [38]:
print("\n--- Travel Plan ---")
print("Destination Overview:\n", results["destination_report"])
print("\n Attractions:\n", results["attractions"])
print("\n Itinerary:\n", results["itinerary"])
print("\n Local Guide:\n", results["guide"])


--- Travel Plan ---
Destination Overview:
 Best Seasons to Visit:
The best seasons to visit Chicago are in the spring (April to June) and fall (September to October) when the weather is mild and the city is alive with outdoor events and activities. Summer (July to August) is also a popular time to visit, but it can be hot and humid. Winter (December to February) can be cold and snowy, but the city is still bustling with holiday events.

Safety:
Chicago, like any major city, has its share of crime, but as a tourist, you can stay safe by being aware of your surroundings, avoiding high-crime areas, and using common sense. Stick to well-lit, populated areas, especially at night, and keep your belongings secure.

Etiquette:
In Chicago, it is important to be polite and respectful to others. When dining out, it is customary to leave a 15-20% tip for good service. It is also polite to say "please" and "thank you" when interacting with locals.

Costs:
Chicago can be an expensive city to visit,

Supervisor with tools Architecture - A supervisor agent uses a tool-calling agent to decide which tool to use.